In [1]:
from neo4j import __version__ as neo4j_version
import pyodbc
import pandas as pd
from neo4j import GraphDatabase
from pandas import DataFrame
import numpy as np
import pandas as pd

In [2]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [3]:
databasePath = r"C:\Users\piyus\BigData\Class Project\Metadata - Class Project\Metadata - Class Project\DataBases.xlsx"
attributeDatatypePath = r"C:\Users\piyus\BigData\Class Project\Metadata - Class Project\Metadata - Class Project\AttributeDataTypes.xlsx"

def loadMetaData(userName, password, groupNumber, databaseName, businessTermPath):
     Sequence = groupNumber * 1000
     conn = Neo4jConnection(uri="bolt://localhost:7687", user = userName, pwd = password)
     print("Connection with Neo4j Established")
     query_string = "CALL apoc.meta.data();"
     dtf_data = DataFrame([dict(_) for _ in conn.query(query_string, db='neo4j')])

     AttributeDataType_table = pd.read_excel(attributeDatatypePath)
     AttributeDataType_table.insert(loc=0, column='AttributeDatTypeId', value=np.arange(101, 101+len(AttributeDataType_table)))
     AttributeDataType_table['PropertyTypes'] = AttributeDataType_table['PropertyTypes'].str.upper()
     # Attribute Table
     attribute_table = dtf_data.iloc[:, [1,6,0,-1]].copy()
     attribute_table.insert(loc=2,column='DatabaseName',value = databaseName)
     attribute_table = attribute_table[(attribute_table.elementType == 'node') & (attribute_table.type != 'RELATIONSHIP')]
     attribute_ta = attribute_table
     attribute_ta.insert(loc=0, column='AttributeId', value=np.arange(Sequence, Sequence+len(attribute_table)))
     attribute_ta = attribute_ta.drop(['elementType'], axis=1)

     df3=pd.merge(attribute_ta,AttributeDataType_table, left_on='type', right_on='PropertyTypes')
     df3= df3.iloc[:,:6]

     pd.to_datetime("today").strftime("%m/%d/%Y")
     label_string = "CALL db.labels();"
     dtf_data_entity = DataFrame([dict(_) for _ in conn.query(label_string, db='neo4j')])
     # Entity Table
     Entity=dtf_data_entity.iloc[:, [0]].copy()
     Entity.insert(loc=0, column='EntityId', value=np.arange(Sequence, Sequence+len(Entity)))
     Entity.insert(loc=2, column='EntityCreationDate', value=pd.to_datetime("today").strftime("%m/%d/%Y"))
     Entity.insert(loc=3, column='Entity_desc', value=Entity.label)
     Entity.insert(loc=4, column='Entity_createdby', value='Group ' + str(groupNumber))
     Entity.insert(loc=5, column='DatabseName', value = databaseName)
     Entity.rename({'label': 'EntityName'}, axis=1, inplace=True)

     # Relation Table
     relation_table = dtf_data.iloc[:, [1, 0,6, 13, -1]].copy()
     # Filter relationships
     relation_table = relation_table[(relation_table.elementType == 'node') & (relation_table.type == 'RELATIONSHIP')]
     Relation=relation_table.iloc[:, [1, 0, 3, -1]].copy()
     Relation = Relation[(Relation.elementType == 'node')]
     Relation.insert(loc=0, column='RelationID', value=np.arange(Sequence, Sequence + len(Relation)))
     Relation['other'] = Relation['other'].str.get(0)
     Relation = Relation.drop(['elementType'], axis=1)
     Relation.rename({'label': 'EntityName1','property' : 'Relationship', 'other':'EntityName2'}, axis=1, inplace=True)

     Database = pd.read_excel(databasePath)
     BussinessTermTable = pd.read_excel(businessTermPath)
     BussinessTermTable.rename({'Column Names': 'BusinessTerms','Column Description' : 'BusinessTermsDescription', 'DataType':'DataType'}, axis=1, inplace=True)
     BussinessTermTable.insert(loc=0, column='BusinessID', value=np.arange(Sequence, Sequence + BussinessTermTable.shape[0]))
     cnxn = pyodbc.connect('Driver={SQL Server Native Client 11.0};'
                           'Server=localhost;'
                           'Database=Class_MetaData;'
                           'Trusted_Connection=yes;')

     cursor = cnxn.cursor()

     cursor.execute("Truncate Table [dbo].[Database];")
     cursor.execute("Truncate Table [dbo].[AttributeDataType];")

     # Insert Dataframe into SQL Server:
     #Common for all Databases
     for index, row in Database.iterrows():
          cursor.execute("INSERT INTO dbo.[Database] (DatabaseId, DatabaseName, DbOwner, DbSize, DbCreationDate, NoofUsers) values(?,?,?,?,?,?);", row.DatabaseId, row.DatabaseName, row.DbOwner, row.DbSize, row.DBCreationDate, row.NumofUsers)
     print('Databases Table Inserted')
     for index, row in AttributeDataType_table.iterrows():
          cursor.execute("INSERT INTO dbo.AttributeDataType (AttributeDataTypeId, AttributeDataTypeValue, AttributeDataTypeRange) values(?,?,?);", row.AttributeDatTypeId, row.PropertyTypes, row.Range)
     print('Attribute Data Type Table Inserted')
     #Different for all Databases
     for index, row in df3.iterrows():
          cursor.execute("INSERT INTO dbo.Attribute (AttributeId,AttributeName,AttributeType,databaseName,EntityName,AttributeDataType_AttributeDataTypeId) values(?,?,?,?,?,?)", row.AttributeId, row.property, row.type, row.DatabaseName, row.label, row.AttributeDatTypeId)
     print('Attribute Table Inserted')
     for index, row in Entity.iterrows():
          cursor.execute("INSERT INTO dbo.Entitiy (EntitiyId, EntitiyName, EntitiyCreationDate, EntitiyDescription, EntitiyCreatedBy, DatabaseName) values(?,?,?,?,?,?)", row.EntityId, row.EntityName, row.EntityCreationDate, row.Entity_desc, row.Entity_createdby, row.DatabseName)
     print('Entity Table Inserted')
     for index, row in Relation.iterrows():
          cursor.execute("INSERT INTO dbo.Relations (RelationID, EntityName1, Relation, EntityName2) values(?,?,?,?)", row.RelationID, row.EntityName1, row.Relationship, row.EntityName2)
     print('Relation Table Inserted')
     for index, row in BussinessTermTable.iterrows():
          cursor.execute("INSERT INTO dbo.BusinessTerm (BusinessID, BusinessTerms, BusinessTermsDescription, DataType) values(?,?,?,?)", row.BusinessID, row.BusinessTerms, row.BusinessTermsDescription, row.DataType)
     print('Business Term table Inserted')
     print('Ingestion Done')
     cnxn.commit()
     cursor.close()

In [6]:
businessTermPath = r"C:\Users\piyus\BigData\Class Project\Metadata - Class Project\Metadata - Class Project\BusinessTermListG8.xlsx"
loadMetaData(userName = "admin", 
            password = "admin", 
            groupNumber = 8, 
            databaseName = "Medium EDA", 
            businessTermPath = businessTermPath)

Connection with Neo4j Established
Databases Table Inserted
Attribute Data Type Table Inserted
Attribute Table Inserted
Entity Table Inserted
Relation Table Inserted
Business Term table Inserted
Ingestion Done
